In [1]:
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import category_encoders as ce
import pickle
from sklearn.svm import SVC
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
train = pd.read_csv("../data/sample_train.txt", delimiter="\t")
test = pd.read_csv("../data/test_id.txt", delimiter="\t")

In [3]:
df = pd.concat([train, test], axis=0)

In [4]:
risk = pd.read_csv("../features/risk/risk.csv")
df = df.merge(risk, on="id", how="left")

In [5]:
symbol = pd.read_csv("../features/symbol/symbol.csv")
df = df.merge(symbol, on="id", how="left")

In [6]:
# dp_emb = pd.read_csv("../features/graph/graph_filtered_64.emb")
# df = df.merge(dp_emb, on="id", how="left")

In [7]:
dp_emb = pd.read_csv("../features/graph/deepwalk_128_filtered.emb")
df = df.merge(dp_emb, on="id", how="left")

In [8]:
dp_emb.shape

(28442, 129)

In [9]:
# nv_emb = pd.read_csv("../data/edge/node2vec_64.emb")
# df = df.merge(nv_emb, on="id", how="left")

In [10]:
graph_info = pd.read_csv("../features/graph/graph_info.csv")
df = df.merge(graph_info, on="id", how="left")

In [11]:
# app_pca = pd.read_csv("../data/app/app_pca_16.csv")
# df = df.merge(app_pca, on="id", how="left")

app_lda = pd.read_csv("../features/app/app_lda_16.csv")
df = df.merge(app_lda, on="id", how="left")

# app_lda = pd.read_csv("../data1/app/app_lda_16.csv")
# df = df.merge(app_lda, on="id", how="left")

# app_nmf = pd.read_csv("../data/app/app_nmf_16.csv")
# df = df.merge(app_nmf, on="id", how="left")

In [12]:
# app_info = pd.read_csv("../features/app/app_info.csv")
# df = df.merge(app_info, on="id", how="left")

app_graph

In [13]:
# to_pca = pd.read_csv("../features/app_graph/to_app_pca_mean.csv")
# from_pca = pd.read_csv("../features/app_graph/from_app_pca_mean.csv")

# df = df.merge(to_pca, on="id", how="left")
# df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_num_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_num_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_weight_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_weight_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

In [14]:
# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_num_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_num_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_weight_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_weight_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

In [15]:
# to_lda = pd.read_csv("../features/app_graph/to_app_lda_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_num_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_num_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_weight_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_weight_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

risk_graph

In [16]:
# to_risk = pd.read_csv("../features/risk_graph/to_risk_mean.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_mean.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

# to_risk = pd.read_csv("../features/risk_graph/to_risk_sum.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_sum.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

symbol_graph

In [17]:
# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_sum.csv")
# from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_sum.csv")

# df = df.merge(to_cat_weight, on="id", how="left")
# df = df.merge(from_cat_weight, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_mean.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_mean.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_mean.csv")
from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_mean.csv")

df = df.merge(to_cat_num, on="id", how="left")
df = df.merge(from_cat_num, on="id", how="left")

to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_mean.csv")
from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_mean.csv")

df = df.merge(to_cat_weight, on="id", how="left")
df = df.merge(from_cat_weight, on="id", how="left")

knn

In [18]:
# dp_knn = pd.read_csv("../features/knn/dp_knn_3.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_10.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_30.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_100.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_300.csv")
# df = df.merge(dp_knn, on="id", how="left")

中心性

In [19]:
pr = pd.read_csv("../features/graph/pagerank.csv")
df = df.merge(pr, on="id", how="left")

In [20]:
for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_pagerank%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

In [21]:
hits = pd.read_csv("../features/graph/hits.csv")
df = df.merge(hits, on="id", how="left")

for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_hits%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

In [22]:
# 联系人个数
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["", "_num", "_weight"]:
            for f in ["sum", "mean"]:
                a = pd.read_csv("../features/graph/%s_d2_%s%s_%s.csv" % (d1, d2, t, f))
                df = df.merge(a, on="id", how="left")

In [23]:
# 二度联系人PageRank值
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["num", "weight"]:
            for f in ["sum", "mean"]:
                a = pd.read_csv("../features/graph/%s_pg_%s_%s_%s_%s_%s.csv" % (d1, d2, t, f, t, f))
                df = df.merge(a, on="id", how="left")

In [24]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [25]:
def  process_feature(train_x, valid_x, test_df):
    result = []
    drop_cols = ['id','label']
    for df in [train_x, valid_x, test_df]:
        result.append(df.drop(drop_cols, axis=1))
    return result 

In [26]:
import os
def cv(df, num_folds, param, model_dir, stratified=True, debug=False):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    train_df = df[df.label.notnull()]
    test_df = df[df.label.isnull()]
    seed = 178
    if "seed" in param: 
        seed = param["seed"]
        del param['seed']
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    oof_preds = np.zeros(train_df.shape[0])
    all_test_preds = []    
    feature_importance_df = pd.DataFrame()
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_df['label'])):
        train_x, train_y = train_df.iloc[train_idx], train_df['label'].iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], train_df['label'].iloc[valid_idx]
        fold_preds = test_df[["id"]]
        
        train_x, valid_x, test = process_feature(train_x, valid_x, test_df)
        if n_fold == 0:
            print(train_x.shape, valid_x.shape, test.shape)
        
        train_data = lgb.Dataset(train_x, label=train_y)
        validation_data = lgb.Dataset(valid_x, label=valid_y)

        clf=lgb.train(params,
                      train_data,
                      num_boost_round=10000,
                      valid_sets=[train_data, validation_data],
                      valid_names=["train", "valid"],
                      early_stopping_rounds=100,
                      verbose_eval=100)

        valid_preds = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_preds = clf.predict(test, num_iteration=clf.best_iteration)

        fold_preds['prob'] = test_preds
        fold_preds['fold_id'] = n_fold + 1
        all_test_preds.append(fold_preds)
        
        oof_preds[valid_idx] = valid_preds
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, valid_preds)))
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    full_auc = roc_auc_score(train_df['label'], oof_preds)
    print('Full AUC score %.6f' % full_auc)    
    
    if not debug:
        train_df["prob"] = oof_preds
        train_df[['id', 'prob']].to_csv(model_dir + "pred_train.csv", index= False)

        all_test_preds = pd.concat(all_test_preds, axis=0)
        all_test_preds.to_csv(model_dir + "all_test_preds.csv", index=False)
        
        sub = pd.DataFrame()
        sub['id'] = all_test_preds.id.unique()
        sub.set_index("id", inplace=True)
        sub["prob"] = all_test_preds.groupby("id").prob.mean()
        sub.reset_index().to_csv(model_dir + "sub.txt", index=False)
    return full_auc

In [30]:
params_list =[
 {'boosting_type': 'goss', 'colsample_bytree': 0.8597707839893032, 'learning_rate': 0.020137987253631827, 'max_bin': 680, 'metric': 'auc', 'min_child_weight': 15.13137429865887, 'num_leaves': 148, 'reg_alpha': 24.755502196617275, 'reg_lambda': 473.22036423872703, 'subsample': 1.0, 'seed': 8266},
 {'boosting_type': 'goss', 'colsample_bytree': 0.7461423089790142, 'learning_rate': 0.022403634316008503, 'max_bin': 720, 'metric': 'auc', 'min_child_weight': 13.722096849427011, 'num_leaves': 139, 'reg_alpha': 31.118880469117745, 'reg_lambda': 568.7316959369789, 'subsample': 1.0, 'seed': 1007},
 {'boosting_type': 'goss', 'colsample_bytree': 0.7711273100062909, 'learning_rate': 0.02170945543480002, 'max_bin': 800, 'metric': 'auc', 'min_child_weight': 16.591958642290347, 'num_leaves': 162, 'reg_alpha': 25.265442700587993, 'reg_lambda': 523.961409668332, 'subsample': 1.0, 'seed': 9249},
 {'boosting_type': 'goss', 'colsample_bytree': 0.8293567622583214, 'learning_rate': 0.019674705718624015, 'max_bin': 380, 'metric': 'auc', 'min_child_weight': 14.831746660625141, 'num_leaves': 158, 'reg_alpha': 36.89998579060101, 'reg_lambda': 562.022841877543, 'subsample': 1.0, 'seed': 4516},
 {'boosting_type': 'goss', 'colsample_bytree': 0.7975959829459766, 'learning_rate': 0.022712332085415846, 'max_bin': 840, 'metric': 'auc', 'min_child_weight': 12.849857867023385, 'num_leaves': 170, 'reg_alpha': 31.197984780237, 'reg_lambda': 594.6493602797656, 'subsample': 1.0, 'seed': 1656}
]
result = []
for i, params in enumerate(params_list):
    model_dir = "../model_output/random/lgb_572/%d/" % i
    result.append(cv(df, 5, params, model_dir))

(15166, 572) (3793, 572) (6000, 572)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.897873	valid's auc: 0.693675
[200]	train's auc: 0.944423	valid's auc: 0.70512
[300]	train's auc: 0.969357	valid's auc: 0.70942
[400]	train's auc: 0.982582	valid's auc: 0.709334
[500]	train's auc: 0.990457	valid's auc: 0.706125
Early stopping, best iteration is:
[407]	train's auc: 0.983303	valid's auc: 0.710911


/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.710911
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.893115	valid's auc: 0.683558
[200]	train's auc: 0.945307	valid's auc: 0.691268
[300]	train's auc: 0.970815	valid's auc: 0.69589
[400]	train's auc: 0.984757	valid's auc: 0.697394
[500]	train's auc: 0.991601	valid's auc: 0.702428
[600]	train's auc: 0.995199	valid's auc: 0.701158
Early stopping, best iteration is:
[505]	train's auc: 0.991866	valid's auc: 0.702627
Fold  2 AUC : 0.702627
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.891492	valid's auc: 0.679294
[200]	train's auc: 0.942471	valid's auc: 0.698115
[300]	train's auc: 0.969772	valid's auc: 0.702877
[400]	train's auc: 0.984711	valid's auc: 0.709633
[500]	train's auc: 0.991791	valid's auc: 0.709878
Early stopping, best iteration is:
[480]	train's auc: 0.990819	valid's auc: 0.711063
Fold  3 AUC : 0.711063
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.89

/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(15166, 572) (3793, 572) (6000, 572)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.884404	valid's auc: 0.676774
[200]	train's auc: 0.931685	valid's auc: 0.692924
[300]	train's auc: 0.960208	valid's auc: 0.696344
[400]	train's auc: 0.97658	valid's auc: 0.694175
Early stopping, best iteration is:
[327]	train's auc: 0.965889	valid's auc: 0.697984
Fold  1 AUC : 0.697984
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.881503	valid's auc: 0.678382
[200]	train's auc: 0.930783	valid's auc: 0.685038
[300]	train's auc: 0.958619	valid's auc: 0.691668
[400]	train's auc: 0.97612	valid's auc: 0.694957
Early stopping, best iteration is:
[387]	train's auc: 0.974426	valid's auc: 0.695538
Fold  2 AUC : 0.695538
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.87425	valid's auc: 0.675876
[200]	train's auc: 0.926304	valid's auc: 0.694514
[300]	train's auc: 0.957855	valid's auc: 0.698483
[400]	train

[200]	train's auc: 0.92986	valid's auc: 0.711914
[300]	train's auc: 0.957426	valid's auc: 0.717267
[400]	train's auc: 0.975803	valid's auc: 0.717521
[500]	train's auc: 0.985253	valid's auc: 0.719978
[600]	train's auc: 0.990543	valid's auc: 0.720273
Early stopping, best iteration is:
[556]	train's auc: 0.988903	valid's auc: 0.72162
Fold  5 AUC : 0.721620
Full AUC score 0.699347


In [31]:
result

[0.70163694783048114,
 0.69999173823303595,
 0.6997584641780964,
 0.699414741867048,
 0.69934684343740094]

In [31]:
# params_list = [
#  {'boosting_type': 'goss', 'colsample_bytree': 0.8195238802715543, 'learning_rate': 0.01594020237248668, 'max_bin': 280, 'metric': 'auc', 'min_child_weight': 5.0568419205711495, 'num_leaves': 103, 'reg_alpha': 11.048237063283961, 'reg_lambda': 841.529796155335, 'subsample': 1.0},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.8041329700354292, 'learning_rate': 0.01731533071365025, 'max_bin': 220, 'metric': 'auc', 'min_child_weight': 3.2566822394373447, 'num_leaves': 103, 'reg_alpha': 18.17731826422929, 'reg_lambda': 882.4097077796198, 'subsample': 1.0},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7564616309157163, 'learning_rate': 0.016185025763304193, 'max_bin': 220, 'metric': 'auc', 'min_child_weight': 1.384961272012771, 'num_leaves': 97, 'reg_alpha': 10.575292424478043, 'reg_lambda': 796.2457303576319, 'subsample': 1.0},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7563206500364532, 'learning_rate': 0.01623247105518354, 'max_bin': 220, 'metric': 'auc', 'min_child_weight': 3.522701763813738, 'num_leaves': 111, 'reg_alpha': 12.472041876219095, 'reg_lambda': 824.4818224081879, 'subsample': 1.0},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.815051867967645, 'learning_rate': 0.01624398132791771, 'max_bin': 280, 'metric': 'auc', 'min_child_weight': 4.1806939038801305, 'num_leaves': 96, 'reg_alpha': 13.581340582781785, 'reg_lambda': 786.6050401929367, 'subsample': 1.0}
# ]
# result = []
# for i, params in enumerate(params_list):
#     model_dir = "../model_output/lgb_572/%d/" % i
#     result.append(cv(df, 5, params, model_dir))

(15166, 572) (3793, 572) (6000, 572)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.864346	valid's auc: 0.663371
[200]	train's auc: 0.914227	valid's auc: 0.681743
[300]	train's auc: 0.944195	valid's auc: 0.688829
[400]	train's auc: 0.963207	valid's auc: 0.692564
[500]	train's auc: 0.9764	valid's auc: 0.695614
[600]	train's auc: 0.98465	valid's auc: 0.696871
[700]	train's auc: 0.989634	valid's auc: 0.696676
Early stopping, best iteration is:
[630]	train's auc: 0.986557	valid's auc: 0.698752


/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.698752
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.868392	valid's auc: 0.667105
[200]	train's auc: 0.915714	valid's auc: 0.682764
[300]	train's auc: 0.943856	valid's auc: 0.688222
[400]	train's auc: 0.96272	valid's auc: 0.692619
[500]	train's auc: 0.976434	valid's auc: 0.693439
Early stopping, best iteration is:
[487]	train's auc: 0.97504	valid's auc: 0.694412
Fold  2 AUC : 0.694412
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.86412	valid's auc: 0.700942
[200]	train's auc: 0.896724	valid's auc: 0.709341
[300]	train's auc: 0.915556	valid's auc: 0.714804
[400]	train's auc: 0.924273	valid's auc: 0.716526
[500]	train's auc: 0.943004	valid's auc: 0.718604
[600]	train's auc: 0.95987	valid's auc: 0.721506
Early stopping, best iteration is:
[583]	train's auc: 0.957957	valid's auc: 0.721776
Fold  3 AUC : 0.721776
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.86273

/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(15166, 572) (3793, 572) (6000, 572)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.859042	valid's auc: 0.656298
[200]	train's auc: 0.907246	valid's auc: 0.678224
[300]	train's auc: 0.936528	valid's auc: 0.686577
[400]	train's auc: 0.957617	valid's auc: 0.692899
[500]	train's auc: 0.971313	valid's auc: 0.693194
Early stopping, best iteration is:
[430]	train's auc: 0.962393	valid's auc: 0.694559
Fold  1 AUC : 0.694559
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.859356	valid's auc: 0.665334
[200]	train's auc: 0.905012	valid's auc: 0.683875
[300]	train's auc: 0.934214	valid's auc: 0.68993
[400]	train's auc: 0.955753	valid's auc: 0.692574
[500]	train's auc: 0.970796	valid's auc: 0.693528
[600]	train's auc: 0.979954	valid's auc: 0.693864
Early stopping, best iteration is:
[578]	train's auc: 0.978307	valid's auc: 0.694862
Fold  2 AUC : 0.694862
Training until validation scores don't improve for 100 rounds.
[100]	tra

[100]	train's auc: 0.865654	valid's auc: 0.668677
[200]	train's auc: 0.912936	valid's auc: 0.682633
[300]	train's auc: 0.94303	valid's auc: 0.687897
[400]	train's auc: 0.963415	valid's auc: 0.690029
[500]	train's auc: 0.97618	valid's auc: 0.691312
[600]	train's auc: 0.984832	valid's auc: 0.692722
Early stopping, best iteration is:
[582]	train's auc: 0.983674	valid's auc: 0.693948
Fold  2 AUC : 0.693948
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.864757	valid's auc: 0.698406
[200]	train's auc: 0.897964	valid's auc: 0.70682
[300]	train's auc: 0.913715	valid's auc: 0.709711
[400]	train's auc: 0.924678	valid's auc: 0.71232
[500]	train's auc: 0.942709	valid's auc: 0.713791
Early stopping, best iteration is:
[440]	train's auc: 0.932998	valid's auc: 0.714381
Fold  3 AUC : 0.714381
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.858031	valid's auc: 0.651582
[200]	train's auc: 0.912336	valid's auc: 0.671307
[300]	train'

In [32]:
result

[0.70163694783048114,
 0.69999173823303595,
 0.6997584641780964,
 0.699414741867048,
 0.69934684343740094]